In [32]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split, DataLoader
from torch import nn, optim
from sklearn.metrics import accuracy_score, precision_score, f1_score
from tqdm import tqdm
import os
import numpy as np
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [33]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)

g = torch.Generator().manual_seed(42)

train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

trainset, valset, testset = random_split(dataset, [train_size, val_size, test_size],
                                         generator=g)

train_loader = DataLoader(trainset, batch_size=64, shuffle=True)
val_loader = DataLoader(valset, batch_size=64, shuffle=False)
test_loader = DataLoader(testset, batch_size=64, shuffle=False)

In [34]:
class Perceptron(nn.Module):
    def __init__(self,
                 input_dim: int = 32 * 32 * 3,
                 hidden_dim: int = 128, output_dim: int = 10
                ):
        super().__init__()

        self.activation = nn.ReLU()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        return x

In [35]:
class ModelTrainer:
    def __init__(
        self,
        model,
        device,
        optimizer,
        criterion,
        train_loader,
        val_loader,
        test_loader,
        epochs: int = 1,
        save_weights: bool =True,
        log_dir: str = "runs"
    ):
        self.model = model
        self.device = device
        self.model = self.model.to(self.device)
        self.optimizer = optimizer
        self.criterion = criterion
        
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        
        self.epochs = epochs
        self.epoch = 1

        self.save_weights = save_weights

        self.current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        self.model_name = self.model.__class__.__name__
        self.log_dir = os.path.join("runs", f"{self.model_name}_PerceptronClf_{self.current_time}")

        self.writer = SummaryWriter(log_dir=self.log_dir)

        self.schedular = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode="min",
            factor=0.5,
            patience=2
        )
        
    def process_model(self):
        for epoch in range(self.epochs):
            train_loss, train_accuracy = self._train_model()
            val_loss, val_accuracy = self._validate_model()

            self.schedular.step(val_loss)
            
            self.epoch += 1

        self.writer.close()
        return train_loss, val_loss, train_accuracy, val_accuracy

    def _train_model(self):
        self.model.train()
        total_loss = 0.0
        all_preds, all_targets = [], []

        for images, labels in tqdm(self.train_loader):
            images, labels = images.to(self.device), labels.to(self.device)

            outputs = self.model(images)
            loss = self.criterion(outputs, labels)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())

        train_loss = total_loss / len(self.train_loader)
        accuracy = accuracy_score(all_targets, all_preds)
        precision = precision_score(all_targets, all_preds, average="weighted", zero_division=0)
        f1 = f1_score(all_targets, all_preds, average="weighted", zero_division=0)
        
        self._print_metrics("Train", train_loss, accuracy, precision, f1)
        
        self.writer.add_scalar("Loss/Train", train_loss, self.epoch)
        self.writer.add_scalar("Accuracy/Train", accuracy, self.epoch)
        self.writer.add_scalar("Precision/Train", precision, self.epoch)
        self.writer.add_scalar("F1/Train", f1, self.epoch)

        return train_loss, accuracy

    def _validate_model(self):
        self.model.eval()
        total_loss = 0.0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for images, labels in tqdm(self.val_loader):
                images, labels = images.to(self.device), labels.to(self.device)

                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                total_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(labels.cpu().numpy())
            
        val_loss = total_loss / len(self.val_loader)
        accuracy = accuracy_score(all_targets, all_preds)
        precision = precision_score(all_targets, all_preds, average="weighted", zero_division=0)
        f1 = f1_score(all_targets, all_preds, average="weighted", zero_division=0)
        
        self._print_metrics("Val", val_loss, accuracy, precision, f1)
        
        self.writer.add_scalar("Loss/Val", val_loss, self.epoch)
        self.writer.add_scalar("Accuracy/Val", accuracy, self.epoch)
        self.writer.add_scalar("Precision/Val", precision, self.epoch)
        self.writer.add_scalar("F1/Val", f1, self.epoch)

        return val_loss, accuracy

    def test_model(self):
        self.model.eval()
        total_loss = 0.0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for images, labels in tqdm(self.test_loader):
                images, labels = images.to(self.device), labels.to(self.device)

                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                total_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(labels.cpu().numpy())
                
        test_loss = total_loss / len(self.test_loader)
        accuracy = accuracy_score(all_targets, all_preds)
        precision = precision_score(all_targets, all_preds, average="weighted", zero_division=0)
        f1 = f1_score(all_targets, all_preds, average="weighted", zero_division=0)
        
        self._print_metrics("Test", test_loss, accuracy, precision, f1)

        if self.save_weights:
            torch.save(self.model.state_dict(), 'densora_gru_weights.pth')

        return test_loss, accuracy

    def _print_metrics(self, phase: str, loss, accuracy, precision, f1):
        if phase == "Test":
            print(f"{phase} Loss {self.epoch - 1}: {loss:.4f} | Accuracy: {accuracy:.4f}")
        else:
            print(f"{phase} Loss {self.epoch}: {loss:.4f} | Accuracy: {accuracy:.4f}")

        print(f"Precision: {precision:.4f}")
        print(f"F1 score: {f1:.4f}")

In [36]:
model = Perceptron()
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trainer = ModelTrainer(
    model=model,
    device=device,
    optimizer=optimizer,
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    epochs=3,   
    save_weights=False
)

train_loss, val_loss, train_acc, val_acc = trainer.process_model()
print(f"Final Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
print(f"Final Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

test_loss, test_acc = trainer.test_model()
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:06<00:00, 70.14it/s]


Train Loss 1: 2.2265 | Accuracy: 0.2072
Precision: 0.1802
F1 score: 0.1481


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:01<00:00, 113.30it/s]


Val Loss 1: 2.1698 | Accuracy: 0.2308
Precision: 0.2248
F1 score: 0.1548


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:04<00:00, 110.11it/s]


...

Train Loss 3: 2.0714 | Accuracy: 0.2852
Precision: 0.2655
F1 score: 0.2387


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:01<00:00, 123.58it/s]


Val Loss 3: 2.0352 | Accuracy: 0.2924
Precision: 0.2873
F1 score: 0.2588
Final Train Loss: 2.0714, Val Loss: 2.0352
Final Train Acc: 0.2852, Val Acc: 0.2924


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:01<00:00, 128.16it/s]

Test Loss 3: 2.0363 | Accuracy: 0.2961
Precision: 0.2916
F1 score: 0.2631
Test Loss: 2.0363, Test Accuracy: 0.2961


In [37]:
from tensorboard.backend.event_processing import event_accumulator

ea = event_accumulator.EventAccumulator("runs/Perceptron_PerceptronClf_2025-08-21_08-43-51")
ea.Reload()

print(ea.Tags())  # должно показать: 'scalars'
print(ea.Scalars('Loss/Train'))

{'images': [], 'audio': [], 'histograms': [], 'scalars': ['Loss/Train', 'Accuracy/Train', 'Precision/Train', 'F1/Train', 'Loss/Val', 'Accuracy/Val', 'Precision/Val', 'F1/Val'], 'distributions': [], 'tensors': [], 'graph': False, 'meta_graph': False, 'run_metadata': []}
[ScalarEvent(wall_time=1755755038.138462, step=1, value=2.2264561653137207), ScalarEvent(wall_time=1755755043.8927855, step=2, value=2.12925386428833), ScalarEvent(wall_time=1755755049.4660344, step=3, value=2.0714430809020996)]
